In [2]:
import math
import random
from typing import List

import numpy as np
from qiskit_aer import AerSimulator, Aer
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import MCMT
import time
import hashlib
from math import sqrt, pi
from qiskit.quantum_info import Statevector, Operator, DensityMatrix
from qiskit.circuit import (
    Parameter, QuantumCircuit, ClassicalRegister, QuantumRegister
)
from qiskit.primitives import BackendSampler, StatevectorSampler, Sampler
from qiskit.circuit.library import PhaseGate, QFT
import math
import numpy as np
from random import randint
from qiskit.visualization import plot_histogram


In [3]:
def is_power_of_two(n: int) -> bool:
    return (n != 0) and ((n & (n - 1)) == 0)

def next_power_of_two(n: int) -> int:
    return 2 ** (n - 1).bit_length()

def greatest_common_divisor(a: int, b: int) -> int:
    while b != 0:
        a, b = b, a % b
    return a


In [4]:
def modular_exponentiation(base: int, exponent: int, modulus: int) -> int:
    result = 1
    base = base % modulus

    while exponent > 0:
        if exponent % 2 == 1:
            result = (result * base) % modulus
        exponent = exponent // 2
        base = (base * base) % modulus

    return result


In [108]:
def modular_exponentiation_circuit(base, exponent, modulus):
    # Number of qubits needed to represent the base and the result
    n = base.bit_length()

    # Quantum registers for the base, exponent, and ancilla qubits for modular multiplication
    base_reg = QuantumRegister(n, 'base')
    exponent_reg = QuantumRegister(exponent.bit_length(), 'exponent')
    ancilla_reg = QuantumRegister(n, 'ancilla')  # Corrected number of ancilla qubits

    # Classical register for measurement
    classical_reg = ClassicalRegister(n, 'classical')

    # Quantum circuit
    qc = QuantumCircuit(base_reg, exponent_reg, ancilla_reg, classical_reg)

    # Initialize the base register to the value of the base
    for i in range(n):
        if (base >> i) & 1:
            qc.x(base_reg[i])

    # Modular exponentiation
    for i in range(exponent.bit_length()):
        if (exponent >> i) & 1:
            # Perform modular multiplication
            for j in range(n):
                qc.cswap(exponent_reg[i], base_reg[j], ancilla_reg[j])
            # Apply the modular reduction (not explicitly implemented in this simplified version)
            qc.barrier()

    # Measure the result
    qc.measure(base_reg, classical_reg)

    return qc
# Example usage
base = 4
exponent = 2
modulus = 5
qc = modular_exponentiation_circuit(base, exponent, modulus)
qc.draw('mpl')

# Execute the circuit
backend = Aer.get_backend('aer_simulator')
transpiled_circuit = transpile(qc, backend)

# Run the transpiled circuit
job = backend.run(transpiled_circuit, shots=1024)
result = job.result()
counts = result.get_counts(qc)
print("Result of modular exponentiation:", counts)

Result of modular exponentiation: {'100': 1024}


In [109]:
from qiskit.circuit import Gate

def shor_algorithm(a: int, N: int, qft_qubits: int) -> List[int]:
    quantum_circuit = QuantumCircuit(qft_qubits, qft_qubits)

    # Apply Hadamard gates to all qubits
    for qubit in range(qft_qubits):
        quantum_circuit.h(qubit)

    # Apply controlled-U gates
    for qubit in range(qft_qubits):
        control_qubits = [i for i in range(qubit)]
        if not control_qubits:
            quantum_circuit.u(0, math.pi * (2 ** qubit), 0, qubit)
        else:
            quantum_circuit.cp(math.pi * (2 ** qubit), control_qubits, qubit)
        modular_exponentiation_gate = Gate("modular_exponentiation", 1, [], None, modular_exponentiation_circuit(a, 2 ** qubit, N))
        quantum_circuit.append(modular_exponentiation_gate, [qubit])

    # Apply inverse QFT
    quantum_circuit.append(QFT(qft_qubits, do_swaps=False).inverse(), range(qft_qubits))

    # Measure all qubits
    for qubit in range(qft_qubits):
        quantum_circuit.measure(qubit, qubit)

    return quantum_circuit


In [110]:
from qiskit import assemble
from qiskit.circuit import Gate
from qiskit.compiler import transpile
from qiskit.visualization import plot_histogram
from math import gcd
def complete_shor_algorithm(N):
    simulator = Aer.get_backend('aer_simulator')

    # Store found factors in a set to avoid duplicates
    found_factors = set()

    for a in range(2, N):
        if gcd(a, N) != 1:
            continue

        qft_qubits = 2 * N.bit_length()
        num_qubits_mod_exp = modular_exponentiation_circuit(a, 1, N).num_qubits
        total_qubits = max(qft_qubits, num_qubits_mod_exp)
        quantum_circuit = QuantumCircuit(total_qubits, total_qubits)

        mod_exp_gate = modular_exponentiation_circuit(a, 1, N).to_gate(label='mod_exp')
        quantum_circuit.append(mod_exp_gate, range(num_qubits_mod_exp))

        quantum_circuit.measure(range(total_qubits), range(total_qubits))

        transpiled_circuit = transpile(quantum_circuit, simulator, optimization_level=0)
        job = simulator.run(transpiled_circuit, shots=10000)
        result = job.result()
        counts = result.get_counts()

        measurement = max(counts, key=counts.get)
        if int(measurement, 2) == 0:
            continue
        period = int(np.round(qft_qubits / int(measurement, 2)))

        if period % 2 == 0:
            factor1 = gcd(a ** (period // 2) - 1, N)
            factor2 = gcd(a ** (period // 2) + 1, N)
            if factor1 != 1 and factor2 != N and factor1 * factor2 == N:
                found_factors.update([factor1, factor2])

    # If only trivial factors are found, return [1, N]
    if found_factors == {1, N}:
        return [1, N]
    else:
        # Return the non-trivial factors as a sorted list
        return sorted(list(found_factors))

# Test the function
for number in range(2, 39):
    factors = complete_shor_algorithm(number)
    if len(factors) > 1 and factors != [1, number]:
        print(f"{number} = {factors[0]} * {factors[1]}")
    elif len(factors) == 1:
        print(f"{number} = {factors[0]}")
    else:
        print(f"{number} has no non-trivial")



2 has no non-trivial


QiskitError: 'Circuit with classical bits cannot be converted to gate.'

In [103]:
for number in range(2, 39):
    factors = complete_shor_algorithm(number)
    if len(factors) == 2:
        print(f"{number} = {factors[0]} * {factors[1]}")


2 = 1 * 2
3 = 1 * 3
4 = 1 * 4
5 = 1 * 5
6 = 1 * 6
7 = 1 * 7
8 = 1 * 8
9 = 1 * 9
10 = 1 * 10
11 = 1 * 11
12 = 1 * 12
13 = 1 * 13
14 = 1 * 14
15 = 1 * 15
16 = 1 * 16
17 = 1 * 17
18 = 1 * 18
19 = 1 * 19
20 = 1 * 20
21 = 1 * 21
22 = 1 * 22
23 = 1 * 23
24 = 1 * 24
25 = 1 * 25
26 = 1 * 26
27 = 1 * 27
28 = 1 * 28
29 = 1 * 29
30 = 1 * 30
31 = 1 * 31
32 = 1 * 32
33 = 1 * 33
34 = 1 * 34
35 = 1 * 35
36 = 1 * 36
37 = 1 * 37
38 = 1 * 38
